Magic Numbers woah

In [2]:
STARTMONTH = 1
STARTDAY = 30
schedulestr = "THRONE Schedule Fall 2.xlsx"

### breaks
# change for sememster (fall and spring breaks)
BREAKMONTH = 3
BREAKDAY = 6 # monday of break
THANKSGIVINGBREAKMONTH = 11
THANKSGIVINGBREAKDAY = 22 # tuesday of thanksgiving week

# adjust february for leap year
month_days = {9: 30, 10: 31, 11: 30, 12: 31, 1: 31, 2: 28, 3: 31, 4: 30, 5: 31}

Arrange data

In [3]:
''' 
rehearsal schedule (fall 21)

Monday: NS 20:30-22:30 label = 1
Tuesday: NS Warm Up 21:30-23:30 label = 0
Wednesday: NS 23:30-1:30 label = 1
Thursday: Whitman 17:00-19:00 label = 0; Wilcox 23:00-1:00 label = 0
Friday: Bloomberg 14:00-16:00 label = 0; Whitman 21:00-23:00 label = 0
Saturday: Bloomberg 21:00-23:00 label = 0; Whitman 12:00-14:00 label = 0; NS 22:00-24:00 label = 1; NS Warm Up 19:00-21:00 label = 0; NS Theater 14:00-16:00 label = 0
Sunday: Whitman 10:00-12:00 label = 0; NS Warm Up 20:00-22:00 label = 0;

'''

# name : string
# day : int (monday = 1, sunday = 7)
# timestart, timeend : float
# size : int ( 1 if large or 0 if small )
class Rehearsal:
    def __init__(self, name, day, timestart, timeend, size):
        self.name = name
        self.day = day
        self.timestart = timestart
        self.timeend = timeend
        self.size = size

# first week
# rehearsals = []
# rehearsals.append( Rehearsal('NS', 1, 21.5, 23.0, 1) )
# rehearsals.append( Rehearsal('Bloomberg', 1, 23.0, 1.0, 0) )
# rehearsals.append( Rehearsal('Whitman', 2, 17.0, 19.0, 0) )
# rehearsals.append( Rehearsal('NS Warm Up', 3, 21.5, 23.5, 0) )
# rehearsals.append( Rehearsal('NS Warm Up', 4, 23.5, 1.5, 0) )
# rehearsals.append( Rehearsal('NS Warm Up', 5, 22.5, 0.5, 0) )
# rehearsals.append( Rehearsal('NS Warm Up', 6, 12.0, 14.0, 0) )
# rehearsals.append( Rehearsal('NS Warm Up 1', 7, 11.0, 13.0, 0) )
# rehearsals.append( Rehearsal('NS Warm Up 2', 7, 23.5, 1.5, 0) )

# second week
# rehearsals = []
# rehearsals.append( Rehearsal('NS', 1, 21.5, 23.0, 1) )
# rehearsals.append( Rehearsal('Bloomberg', 1, 23.0, 1.0, 0) )
# rehearsals.append( Rehearsal('Whitman', 2, 17.0, 19.0, 0) )
# rehearsals.append( Rehearsal('NS Warm Up', 2, 21.5, 23.0, 0) )
# rehearsals.append( Rehearsal('NS Warm Up', 3, 21.5, 23.5, 0) )
# rehearsals.append( Rehearsal('NS Warm Up', 4, 23.5, 1.5, 0) )
# rehearsals.append( Rehearsal('NS', 5, 20.5, 22.5, 1) )
# rehearsals.append( Rehearsal('NS Warm Up', 5, 22.5, 0.5, 0) )
# rehearsals.append( Rehearsal('NS Warm Up', 6, 12.0, 14.0, 0) )
# rehearsals.append( Rehearsal('Murphy', 6, 21.0, 23.0, 1) )
# rehearsals.append( Rehearsal('NS Warm Up 1', 7, 11.0, 13.0, 0) )
# rehearsals.append( Rehearsal('NS Warm Up 2', 7, 23.5, 1.5, 0) )

# rest of sem
rehearsals = []
rehearsals.append( Rehearsal('NS', 1, 21.0, 23.0, 1) )
rehearsals.append( Rehearsal('Bloomberg 1', 1, 21.0, 23.0, 0) )
rehearsals.append( Rehearsal('Bloomberg 2', 1, 23.0, 1.0, 0) )
rehearsals.append( Rehearsal('Whitman', 2, 17.0, 19.0, 0) )
rehearsals.append( Rehearsal('NS Warm Up', 2, 21.5, 23.0, 0) )
rehearsals.append( Rehearsal('NS Warm Up', 3, 21.5, 23.5, 0) )
rehearsals.append( Rehearsal('NS Warm Up', 4, 23.5, 1.5, 0) )
rehearsals.append( Rehearsal('NS', 5, 20.5, 22.5, 1) )
rehearsals.append( Rehearsal('NS Warm Up', 5, 22.5, 0.5, 0) )
rehearsals.append( Rehearsal('NS Warm Up', 6, 12.0, 14.0, 0) )
rehearsals.append( Rehearsal('Murphy', 6, 21.0, 23.0, 1) )
rehearsals.append( Rehearsal('Bloomberg', 6, 19.0, 21.0, 0) )
rehearsals.append( Rehearsal('NS Warm Up 1', 7, 11.0, 13.0, 0) )
rehearsals.append( Rehearsal('Bloomberg', 7, 13.0, 15.0, 0) )
rehearsals.append( Rehearsal('NS Warm Up 2', 7, 23.5, 1.5, 0) )

### first day of rehos, always a monday

for i in rehearsals:
    print(i.name)


NS
Bloomberg 1
Bloomberg 2
Whitman
NS Warm Up
NS Warm Up
NS Warm Up
NS
NS Warm Up
NS Warm Up
Murphy
Bloomberg
NS Warm Up 1
Bloomberg
NS Warm Up 2


In [4]:
''' 
student conflicts
conflicts represented by 7 arrays one for each day?
'''

class Unique:
    def __init__(self, month, day, interval):
        self.month = month
        self.day = day
        self.interval = interval

# conflicts are arrays of tuples, start and end are the tuple values
class Dancer:
    def __init__(self, name, monday, tuesday, wednesday, thursday, friday, saturday, sunday, other):
        self.name = name
        self.monday = monday
        self.tuesday = tuesday
        self.wednesday = wednesday
        self.thursday = thursday
        self.friday = friday
        self.saturday = saturday
        self.sunday = sunday
        self.other = other

In [5]:
''' 
read spreadsheet
'''

import pandas as pd

data = pd.read_excel("temp.xlsx")

data.head()

,First,Last,Mondays,Tuesdays,Wednesdays,Thursdays,Fridays,Saturdays,Sundays,Notes
0,Anna,Wang,NaN,4:30PM-6:30PM,NaN,7:00PM-11:00PM,NaN,NaN,NaN,10/2.4:00PM-9:00PM;10/5.5:00PM-9:00PM
1,Arec,Keomurjian,4:45PM-6:30PM,4:45PM-6:30PM;9:00PM-11:30PM,4:45PM-6:30PM;9:00PM-11:30PM,4:45PM-6:30PM,4:45PM-6:30PM;8:00PM-10:00PM,10:00AM-1:00PM,NaN,10/9.9:30PM-11:00PM
2,Arra,Ju,NaN,NaN,NaN,NaN,NaN,NaN,7:00PM-10:00PM,10/24.9:00PM-11:00PM;10/26.11:30PM-1:30AM;10/2...
3,Chaeyoung,Lee,7:30PM-9:20PM,4:30PM-5:30PM,NaN,NaN,NaN,NaN,NaN,NaN
4,Diana,Kim,7:30PM-10:20PM,NaN,NaN,NaN,NaN,NaN,7:00PM-10:00PM,11/1.7:30PM-9:30PM;11/8.7:30PM-9:30PM;11/19.5:...


In [6]:
# returns array of tuples (start, end) for each conflict
# input string conflicts should be of the form hh:mm am - hh:mm pm with semicolons in between each conflict 
# spaces optional and am and pm should be correct with respect to the conflict (caps optional)
def conflict_split(str):
    out = []
    str = str.replace(' ', '')
    str = str.upper()
    arr = str.split(";")
    for i in arr:
        conflict = i.split("-")
        startstr = conflict[0]
        endstr = conflict[1]

        colon_index = startstr.index(":")
        starthour = int(startstr[0:colon_index])
        startminute = int(startstr[colon_index+1:colon_index+3])/60
        startampm = startstr[colon_index+3:len(startstr)]
        start = starthour+startminute
        if startampm == "PM" and starthour != 12:
            start += 12
        if startampm == "AM" and starthour == 12:
            start -= 12

        colon_index = endstr.index(":")
        endhour = int(endstr[0:colon_index])
        endminute = int(endstr[colon_index+1:colon_index+3])/60
        endampm = endstr[colon_index+3:len(endstr)]
        end = endhour+endminute
        if endampm == "PM" and endhour != 12:
            end += 12
        if endampm == "AM" and endhour == 12:
            end -= 12

        out.append((start, end))

    return out

conflict_split("12:20pm -2:00PM;4:00PM-6:00PM")

[(12.333333333333334, 14.0), (16.0, 18.0)]

In [7]:
# returns array of Unique objects for each conflict
# input string conflicts should be of the form month / day . hh:mm am - hh:mm pm with semicolons in between each conflict 
# spaces optional and am and pm should be correct with respect to the conflict (caps optional)
def unique_split(str):
    out = []
    str = str.replace(' ', '')
    str = str.upper()
    arr = str.split(";")
    for i in arr:
        i1 = i.index("/")
        i2 = i.index(".")
        month = int(i[0: i1])
        day = int(i[i1+1: i2])
        
        temp = i[i2+1:]
        conflict = temp.split("-")
        startstr = conflict[0]
        endstr = conflict[1]

        colon_index = startstr.index(":")
        starthour = int(startstr[0:colon_index])
        startminute = int(startstr[colon_index+1:colon_index+3])/60
        startampm = startstr[colon_index+3:len(startstr)]
        start = starthour+startminute
        if startampm == "PM" and starthour != 12:
            start += 12
        if startampm == "AM" and starthour == 12:
            start -= 12

        colon_index = endstr.index(":")
        endhour = int(endstr[0:colon_index])
        endminute = int(endstr[colon_index+1:colon_index+3])/60
        endampm = endstr[colon_index+3:len(endstr)]
        end = endhour+endminute
        if endampm == "PM" and endhour != 12:
            end += 12
        if endampm == "AM" and endhour == 12:
            end -= 12
        
        u = Unique(month, day, (start, end))
        out.append(u)
    return out

print(unique_split("12/3.4:00 AM-12:00PM;12/4.5:00AM-11:00PM")[1].interval)

(5.0, 23.0)


In [8]:
# reads all dancer data

all_dancers = []
array_data = data.to_numpy()

for d in array_data:
    name = d[0] + " " + d[1]
    temparr = [[], [], [], [], [], [], []]
    for i in range(7):
        if type(d[i+2]) == type(""):
            temparr[i] = conflict_split(d[i+2])
    monday = temparr[0]
    tuesday = temparr[1]
    wednesday = temparr[2]
    thursday = temparr[3]
    friday = temparr[4]
    saturday = temparr[5]
    sunday = temparr[6]
    other = []
    if type(d[9]) == type(""):
        other = unique_split(d[9])

    s = Dancer(name, monday, tuesday, wednesday, thursday, friday, saturday, sunday, other)
    all_dancers.append(s)
    # check for all dancers
    print(s.name)

Anna Wang
Arec Keomurjian
Arra Ju
Chaeyoung Lee
Diana Kim
Eric Xie
Kendrah Su
Jadi Wang
Jeannie Kim
Rebecca Cho
Andrew Hwang
Annie Liu
Caren Kim
Jaehee Ashley
Joseph Hwang
Keilly Ponce-Merida
Yuri Yu
Alex Ni
Benjamin Liu
Chaewon Han
Diego Solorio
Holly Thompson
Jimmy Hoang
Joy Cho
Michelle Kim
Samantha Lee
Theresa Lim
Yu Kyung Kim
Zhaoran Chen
Angel Osaseri
Ayush Alag
Bella Racioppi
Brian Tieu
Chloe Chen
Cindy Cheng
Claire Liu
Ethan Lam
Ivy Wang
Martin Lee
Mona Hassan
Monica Song
Rachel Qing Pang
Rose Nguyen
Sam Liang
Shannon Heh
Michael Pi


In [9]:
'''
piece :))
'''
# removed size, implementation of it is scuffed atm
class Piece:
    def __init__(self, name, dancers):
        self.name = name
        self.dancers = dancers
        # self.size = size


In [10]:
pieces_data = pd.read_excel(schedulestr)

pieces_names = list(pieces_data.columns)

pieces_data_array = pieces_data.to_numpy().transpose().tolist()

for i in range(len(pieces_data_array)):
    mylist = pieces_data_array[i]
    newlist = [x for x in mylist if type(x) == type("")]
    pieces_data_array[i] = newlist

# print(pieces_data_array)
pieces = []

for i in range(len(pieces_names)):
    name = pieces_names[i]
    dancers = []
    for j in (pieces_data_array[i]):
        for k in all_dancers:
            if k.name == j:
                dancers.append(k)
                break
    # size = 0
    # if len(dancers) > 10 and (name == 'Luna Backdancer (12)' or name == 'BTBT (12)'):
    #     size = 1

    p = Piece(name, dancers)
    pieces.append(p)

# check for correct number of members
for i in pieces:
    print(i.name + ": " + str(len(i.dancers)))



Luna Backdancer (12): 18
Odd Eye Mob (8): 14
BTBT (12): 12
Paint the Town (12): 12
Flash (11): 11
Reveal (11): 11
Trash Backdancer (4): 11
Gotta Go Backdancer (4): 11
Favorite (9): 9
Feel Special (9): 9
Domino (8): 8
Growl (8): 8
Fake Love (7): 7
Fever (7): 7
Odd Eye (7): 7
Trash (7): 7
Gotta Go (5): 7
Luna (6): 6
RUN2U (6): 6
Illusion (6): 6
Rumor (5): 5


NOW THE ACTUAL SCHEDULING :)))

In [11]:
# returns if there is a conflict or not
def is_conflict(time1, time2):
    x1 = time1[0]
    x2 = time1[1]
    y1 = time2[0]
    y2 = time2[1]
    if x2 < x1:
        x2 += 24
    if y2 < y1:
        y2 += 24
    return x1 < y2 and y1 < x2

# given month and day, find week number and day of week
# week 0 is the first week
# 1 is monday, 7 is sunday
def read_date(month, day):
    month_dif = month - STARTMONTH
    day_dif = day - STARTDAY
    if month_dif > 6:
        return -1, -1
    n = 0
    for i in range(month_dif):
        n += month_days[STARTMONTH + i]
    n += day_dif
    weekday = (n % 7) + 1
    weeknum = n // 7
    if (month > BREAKMONTH or (month == BREAKMONTH and day > BREAKDAY)) and (STARTMONTH < BREAKMONTH or (STARTMONTH == BREAKMONTH and STARTDAY < BREAKDAY)):
        weeknum -= 1

    return weeknum, weekday

# returns the number of conflicts with a piece and a rehearsal time
def num_conflicts(rehearsal, piece, weeknum):
    n = 0
    who_conflict = set()
    # if rehearsal.size < piece.size:
    #     n += 1
    x, _ = read_date(THANKSGIVINGBREAKMONTH, THANKSGIVINGBREAKDAY)
    if weeknum == x and rehearsal.day >= 2:
        return -1
    for d in piece.dancers:
        arr = [d.monday, d.tuesday, d.wednesday, d.thursday, d.friday, d.saturday, d.sunday]
        day = rehearsal.day
        conflicts = arr[day - 1]
        rehearsal_time = (rehearsal.timestart, rehearsal.timeend)
        for c in conflicts:
            if is_conflict(rehearsal_time, c):
                n += 1
                who_conflict.add(d.name)
                break
        for u in d.other:
            unum, uday = read_date(u.month, u.day)
            if unum == weeknum:
                if uday == day:
                    if is_conflict(rehearsal_time, u.interval):
                        n += 1
                        who_conflict.add(d.name)
                        break            
    return n, who_conflict

# weekly returns the week of rehearsals and which pieces can fit in which rehos with the given buffer
# buffer is the number of dancers that will not be available for a rehearsal
def weekly(buffer, weeknum):
    possible_rehearsals = {}
    for r in rehearsals:
        arr = []
        for p in pieces:
            n, who_conflict = num_conflicts(r, p, weeknum)
            out = p.name
            if buffer > 0:
                out += ", " + str(who_conflict)
            if n == buffer:
                arr.append(out)
        possible_rehearsals[r.name + " " + str(r.day)] = arr
    return possible_rehearsals
        
weekly(0,1)

{'NS 1': ['RUN2U (6)'],
 'Bloomberg 1 1': ['RUN2U (6)'],
 'Bloomberg 2 1': ['BTBT (12)',
  'Paint the Town (12)',
  'Trash Backdancer (4)',
  'Feel Special (9)',
  'Growl (8)',
  'Fever (7)',
  'Trash (7)',
  'Gotta Go (5)',
  'Luna (6)',
  'RUN2U (6)',
  'Illusion (6)'],
 'Whitman 2': ['Feel Special (9)',
  'Growl (8)',
  'Fever (7)',
  'Trash (7)',
  'Gotta Go (5)',
  'Rumor (5)'],
 'NS Warm Up 2': ['BTBT (12)',
  'Trash Backdancer (4)',
  'Feel Special (9)',
  'Domino (8)',
  'Fake Love (7)',
  'Fever (7)',
  'Trash (7)',
  'Luna (6)',
  'Illusion (6)'],
 'NS Warm Up 3': ['Domino (8)'],
 'NS Warm Up 4': ['Luna Backdancer (12)',
  'BTBT (12)',
  'Flash (11)',
  'Trash Backdancer (4)',
  'Favorite (9)',
  'Feel Special (9)',
  'Domino (8)',
  'Fake Love (7)',
  'Fever (7)',
  'Trash (7)',
  'Luna (6)',
  'RUN2U (6)',
  'Illusion (6)'],
 'NS 5': ['BTBT (12)',
  'Feel Special (9)',
  'Growl (8)',
  'Fever (7)',
  'Odd Eye (7)',
  'Gotta Go (5)',
  'Luna (6)',
  'Illusion (6)',
  'Rumor 

In [12]:
# Given piece, find rehearsal times that work
def find_reho_time(piece_name, weeknum, buffer):
    piece = None
    for p in pieces:
        if piece_name == p.name:
            piece = p
    if piece is None:
        return -1
    possible_rehearsals = []
    for r in rehearsals:
        arr = []
        n, who_conflict = num_conflicts(r, piece, weeknum)
        out = r.name + " " + str(r.day)
        if buffer > 0:
            out += ", " + str(who_conflict)
        if n == buffer:
            possible_rehearsals.append(out)
    return possible_rehearsals

find_reho_time('Feel Special (9)', 1, 0)

['Bloomberg 2 1',
 'Whitman 2',
 'NS Warm Up 2',
 'NS Warm Up 4',
 'NS 5',
 'NS Warm Up 5',
 'Murphy 6',
 'Bloomberg 6',
 'NS Warm Up 1 7']

In [18]:
def find_time(piece_name, weeknum, buffer=0):
    piece = None
    for p in pieces:
        if piece_name == p.name:
            piece = p
    if piece is None:
        return -1

    options = []
    for i in range(1,8):
        if i < 6:
            # 5pm to midnight
            for j in range(34, 48):
                temp = Rehearsal('NS', i, j/2, j/2 + 2, 1)
                n, who_conflict = num_conflicts(temp, piece, weeknum)
                if n == buffer:
                    options.append((i, j/2, j/2 + 2, n))
        if i >= 6:
            # 10am to midnight
            for j in range(20, 48):
                temp = Rehearsal('NS', i, j/2, (j+1)/2, 1)
                n, who_conflict = num_conflicts(temp, piece, weeknum)
                if n == buffer:
                    options.append((i, j/2, j/2 + 2, n))
    return options

find_time('Feel Special (9)', 1)

[(1, 22.5, 24.5, 0),
 (1, 23.0, 25.0, 0),
 (1, 23.5, 25.5, 0),
 (2, 17.0, 19.0, 0),
 (2, 17.5, 19.5, 0),
 (2, 18.0, 20.0, 0),
 (2, 18.5, 20.5, 0),
 (2, 19.0, 21.0, 0),
 (2, 19.5, 21.5, 0),
 (2, 20.0, 22.0, 0),
 (2, 20.5, 22.5, 0),
 (2, 21.0, 23.0, 0),
 (2, 21.5, 23.5, 0),
 (2, 22.0, 24.0, 0),
 (2, 22.5, 24.5, 0),
 (2, 23.0, 25.0, 0),
 (2, 23.5, 25.5, 0),
 (4, 20.5, 22.5, 0),
 (4, 21.0, 23.0, 0),
 (4, 21.5, 23.5, 0),
 (4, 22.0, 24.0, 0),
 (4, 22.5, 24.5, 0),
 (4, 23.0, 25.0, 0),
 (4, 23.5, 25.5, 0),
 (5, 20.5, 22.5, 0),
 (5, 21.0, 23.0, 0),
 (5, 21.5, 23.5, 0),
 (5, 22.0, 24.0, 0),
 (5, 22.5, 24.5, 0),
 (5, 23.0, 25.0, 0),
 (5, 23.5, 25.5, 0),
 (6, 18.0, 20.0, 0),
 (6, 18.5, 20.5, 0),
 (6, 19.0, 21.0, 0),
 (6, 19.5, 21.5, 0),
 (6, 20.0, 22.0, 0),
 (6, 20.5, 22.5, 0),
 (6, 21.0, 23.0, 0),
 (6, 21.5, 23.5, 0),
 (6, 22.0, 24.0, 0),
 (6, 22.5, 24.5, 0),
 (6, 23.0, 25.0, 0),
 (6, 23.5, 25.5, 0),
 (7, 10.0, 12.0, 0),
 (7, 10.5, 12.5, 0),
 (7, 11.0, 13.0, 0),
 (7, 11.5, 13.5, 0),
 (7, 12.0, 14

In [14]:
'''
now time to create full schedule
'''

# dict of piece to number of rehearsals scheduled
piece_rehearsals = {}
for p in pieces:
    piece_rehearsals[p.name] = 0

# print(piece_rehearsals)

In [15]:
# ## REDO OR ABANDON, likely abandon

# def find_champ(iter, week, r, possible_rehearsals):
#     if iter > 3:
#         return "None", -1
#     if len(possible_rehearsals[r]) == 0:
#         possible_rehearsals = weekly(iter + 1, week)
#         return find_champ(iter+1, week, r, possible_rehearsals)
#     else:
#         champ = possible_rehearsals[r][0]
#         champ_val = piece_rehearsals[champ]
#         for p in possible_rehearsals[r]:
#             if piece_rehearsals[p] < champ_val:
#                 champ = p
#                 champ_val = piece_rehearsals[p]
#         if piece_rehearsals[champ] >= max[champ]:
#             possible_rehearsals = weekly(iter + 1, week)
#             return find_champ(iter+1, week, r, possible_rehearsals)
#         else:
#             return champ, iter

# max = {
#     "Luna Backdancer (12)" : 5,
#     "Odd Eye Mob (8)" : 3,
#     "BTBT (12)" : 5,
#     "Paint the Town (12)" : 5,
#     "Flash (11)" : 5,
#     "Reveal (11)" : 5,
#     "Trash Backdancer (4)" : 3,
#     "Gotta Go Backdancer (4)" : 1,
#     "Favorite (9)" : 5,
#     "Feel Special (9)" : 5,
#     "Domino (8)" : 5,
#     "Growl (8)" : 5,
#     "Fake Love (7)" : 5,
#     "Fever (7)" : 5,
#     "Odd Eye (7)" : 3,
#     "Trash (7)" : 3,
#     "Gotta Go (5)" : 2,
#     "Luna (6)" : 0,
#     "RUN2U (6)" : 5,
#     "Illusion (6)" : 5,
#     "Rumor (5)" : 5
#     }

# # num_weeks is the number of weeks to schedule rehearsals
# # min is the minimum number of rehearsals for a piece
# # max is the maximum number of rehearsals for a piece
# def schedule(num_weeks, max):
#     for i in range(num_weeks):
#         possible_rehearsals = weekly(0, i)
#         print("Week " + str(i))
#         # print(possible_rehearsals)
#         for r in possible_rehearsals:
#             # if len(possible_rehearsals[r]) == 0:
#             #     print(r + ": None")
#             # else:
#             #     champ = possible_rehearsals[r][0]
#             #     champ_val = piece_rehearsals[champ]
#             #     for p in possible_rehearsals[r]:
#             #         if piece_rehearsals[p] < champ_val:
#             #             champ = p
#             #             champ_val = piece_rehearsals[p]
#             #     if piece_rehearsals[champ] >= max:
#             #         possible_rehearsals = weekly(1, i)
#             #         champ = possible_rehearsals[r][0]
#             #         champ_val = piece_rehearsals[champ]
#             #         for p in possible_rehearsals[r]:
#             #             if piece_rehearsals[p] < champ_val:
#             #                 champ = p
#             #                 champ_val = piece_rehearsals[p]
#             #         print("missing 1")
#             #         possible_rehearsals = weekly(0, i)
#             champ, missing = find_champ(0, i, r, possible_rehearsals)
#             if champ != "None":
#                 piece_rehearsals[champ] += 1
#             print(r + ": " + champ)
#             if missing > 0:
#                 print("missing " + str(missing))

# schedule(7, max)
# print(piece_rehearsals)

In [16]:
# # find available times

# oldies = [
#     "Caren Kim",
#     "Jaehee Ashley",
#     "Joseph Hwang",
#     "Keilly Ponce-Merida",
#     "Yuri Yu",
#     "Alex Ni",
#     "Benjamin Liu",
#     "Chaewon Han",
#     "Destiny Allen",
#     "Diego Solorio",
#     "Holly Thompson",
#     "Michelle Kim",
#     "Samantha Lee",
#     "Theresa Lim",
#     "Zhaoran Chen"
#     "Angel Osaseri",
#     "Ayush Alag",
#     "Bella Racioppi",
#     "Brian Tieu",
#     "Chloe Chen",
#     "Cindy Cheng",
#     "Claire Liu",
#     "Ethan Lam",
#     "Ivy Wang",
#     "Mona Hassan",
#     "Monica Song",
#     "Rachel Qing Pang",
#     "Rose Nguyen",
#     "Sam Liang",
#     "Shannon Heh"
# ]


# name = 'oldies'
# dancers = []
# for j in (oldies):
#     for k in all_dancers:
#         if k.name == j:
#             dancers.append(k)
#             break

# p = Piece(name, dancers)

# def find(piece, weeknum):
#     options = []
#     for i in range(1,8):
#         if i < 6:
#             #friday to midnight
#             for j in range(34, 48):
#                 temp = Rehearsal('NS', i, j/2, j/2 + 2, 1)
#                 n = num_conflicts(temp, piece, weeknum)
#                 options.append((i, j/2, n))
#                 if n < champnum:
#                     champnum = n
#                     champhour = j/2
#                     champday = i
#         if i >= 6:
#             for j in range(20, 48):
#                 temp = Rehearsal('NS', i, j/2, (j+1)/2, 1)
#                 n = num_conflicts(temp, piece, weeknum)
#                 options.append((i, j/2, n))
#                 if n < champnum:
#                     champnum = n
#                     champhour = j/2
#                     champday = i
#     print(options)
#     return champday, champhour, champnum
    

# find(p, 1)